In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import warnings
warnings.filterwarnings('ignore')
from datetime import date, datetime

In [2]:
titulos = pd.read_csv(r"C:\Users\rodli\OneDrive\Documentos\Mestrado\Arquivos\Titulos_data.csv", sep=';', encoding='utf-8')
redes = pd.read_csv(r"C:\Users\rodli\OneDrive\Documentos\Mestrado\Arquivos\Rede_social.csv", sep=';', encoding='utf-8')

In [3]:
data_inicio = '12-02-2024'
data_inicio_titulo = data_inicio
data_inicio = datetime.strptime(data_inicio, '%d-%m-%Y').date()
data_fim = '16-04-2024'
data_fim_titulo = data_fim
data_fim = datetime.strptime(data_fim, '%d-%m-%Y').date()
titulos['data_corte'] = ''
for ind in range(0, len(titulos)):
    titulos['data_corte'][ind] = datetime.strptime(titulos['Data'][ind], '%d/%m/%Y').date()
titulos = titulos[titulos['data_corte'] >= data_inicio]
titulos = titulos[titulos['data_corte'] <= data_fim]

In [14]:
titulo_arb = titulos
variavel = 'Fluminense x Bragantino'
titulo_arb['Variavel'] = titulo_arb.Título.str.contains(variavel)
titulo_arb_filtrado = titulo_arb.query("Variavel == True")
titulo_arb_filtrado = titulo_arb_filtrado.sort_values(by='Sessões', ascending=False)
titulo_arb_filtrado = titulo_arb_filtrado.reset_index(drop=True)
titulo_arb_agrup = titulo_arb_filtrado
titulo_arb_agrup['agrupado'] = titulo_arb_agrup['Título'] + '-' + titulo_arb_agrup['Data']
titulo_arb_agrup = titulo_arb_agrup.head(30)

fig = px.bar(titulo_arb_agrup, x='agrupado', y='Sessões', title='Consultas que envolvam ' + variavel +  ' de ' + str(data_inicio_titulo) + ' até ' + str(data_fim_titulo), color='Sessões')
fig.update_layout(xaxis_tickangle=-45, height=700)
fig.write_html(r"C:\Users\rodli\OneDrive\Documentos\Mestrado\Arquivos\Graficos\Redes_sociais_data_" + variavel + "_" + str(data_inicio_titulo) + "_" + str(data_fim_titulo) + ".html")
fig.show()

In [5]:
buscas = ['Alianza de Lima', 'Alianza Lima', 'Colo Colo', 'Cerro Portenho', 'Jhon Arias', 'German Cano', 'André']

In [6]:
buscas = sorted(buscas)
rodada = 0
colunas = ['Título', 'Extenso', 'Sessões', 'Usuários', 'Novos usuários', 'Data', 'nome_busca', 'nome']
pag_buscaf = pd.DataFrame(columns=colunas)
for ind in range(0, len(buscas)):
    nome = buscas[ind]
    titulos['nome_busca'] = titulos.Título.str.contains(nome)
    pag_busca1 = titulos.query("nome_busca == True")
    pag_busca1['nome'] = buscas[ind]
    rodada = rodada + 1
    if rodada == 0:
        pag_buscaf = pag_busca1
    else:
        pag_buscaf = pd.concat([pag_buscaf, pag_busca1])
    titulos['nome_busca'] = False
pag_buscaf = pag_buscaf.drop(['Título', 'Usuários', 'Novos usuários', 'nome_busca'], axis=1)
pag_buscaf = pag_buscaf.groupby(['Extenso', 'Data', 'nome'], as_index=False).sum()
pag_buscaf = pag_buscaf.sort_values(by=['Extenso', 'Data', 'nome'], ascending=True)
pag_buscaf = pag_buscaf.reset_index(drop=True)

In [7]:
final = []
array_data = pag_buscaf.Data.unique()
array_ext = pag_buscaf.Extenso.unique()
for ind2 in range(0, len(array_data)):
    for ind in range(0, len(buscas)):
        result = pag_buscaf.loc[pag_buscaf['nome'] == buscas[ind]]
        result = result.loc[result['Data'] == array_data[ind2]]
        if len(result) == 0:
            insert = (array_ext[ind2], array_data[ind2], buscas[ind], 0)
            final.append(insert)
colunas = ['Extenso', 'Data', 'nome', 'Sessões']
pag_buscaf_aux = pd.DataFrame(data=final, columns=colunas)
pag_buscaf = pd.concat([pag_buscaf, pag_buscaf_aux], sort=True)
pag_buscaf = pag_buscaf.sort_values(by=['Extenso', 'Data', 'nome'], ascending=True)
pag_buscaf = pag_buscaf.reset_index(drop=True)

In [8]:
fig = px.line(pag_buscaf, x="Data", y="Sessões", color='nome', title='Acessos por dia de ' + str(data_inicio_titulo) + ' até ' + str(data_fim_titulo), markers=True)
fig.update_layout(xaxis_tickangle=-45)
fig.write_html(r"C:\Users\rodli\OneDrive\Documentos\Mestrado\Arquivos\Graficos\Redes_sociais_acessos_" + str(data_inicio_titulo) + "_" + str(data_fim_titulo) + ".html")
fig.show()

In [9]:
print(redes)

    Indice    Data                              Assunto  \
0        1  18/jan                      Estreia Carioca   
1        2  18/jan                        Volta Redonda   
2        3  19/jan                          Moça Bonita   
3        4  20/jan                            Camisa 20   
4        5  20/jan                           Portuguesa   
5        6  21/jan               Artilheiros Portuguesa   
6        7  22/jan            Artilheiro por adversário   
7        8  23/jan                        Melhores Anos   
8        9  24/jan                       Elcyr Mendonça   
9       10  25/jan                   Audax e 0 derrotas   
10      11  26/jan                          Nova Iguaçu   
11      12  27/jan              Artilheiros Nova Iguaçu   
12      13  29/jan                 Goleadas sobre Bangu   
13      14  30/jan              Sequências contra Bangu   
14      15  31/jan                  Marcão como Técnico   
15      16  01/fev                Evolução contra Bangu 

In [10]:
fig = px.bar(redes, x='Assunto', y='Impressoes', title='Impressões no X(Twitter)', color='Indice')
fig.update_layout(xaxis_tickangle=-45, height=700)
fig.write_html(r"C:\Users\rodli\OneDrive\Documentos\Mestrado\Arquivos\Graficos\Impressoes_X_" + str(data_inicio_titulo) + ".html")
fig.show()

In [11]:
fig = px.bar(redes, x='Assunto', y='Engajamentos', title='Engajamentos no X(Twitter)', color='Indice')
fig.update_layout(xaxis_tickangle=-45, height=700)
fig.write_html(r"C:\Users\rodli\OneDrive\Documentos\Mestrado\Arquivos\Graficos\Engajamentos_X_" + str(data_inicio_titulo) + ".html")
fig.show()

In [12]:
fig = px.bar(redes, x='Assunto', y='Curtidas', title='Curtidas no X(Twitter)', color='Indice')
fig.update_layout(xaxis_tickangle=-45, height=700)
fig.write_html(r"C:\Users\rodli\OneDrive\Documentos\Mestrado\Arquivos\Graficos\Curtidas_X_" + str(data_inicio_titulo) + ".html")
fig.show()

In [13]:
fig = px.bar(redes, x='Assunto', y='Link_click', title='Link Click no X(Twitter)', color='Indice')
fig.update_layout(xaxis_tickangle=-45, height=700)
fig.write_html(r"C:\Users\rodli\OneDrive\Documentos\Mestrado\Arquivos\Graficos\Link_Click_X_" + str(data_inicio_titulo) + ".html")
fig.show()